# Process stackoverflow data

In [12]:
%load_ext autoreload
%autoreload 2

from google.cloud import bigquery
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from general_utils import apply_parallel, flattenlist

EN = spacy.load('en')

In [13]:
use_cache = False

In [14]:
# TODO: use csv for now, fix credential issue
# bigquery
# https://console.cloud.google.com/bigquery?sq=784649123063:df10117d54ab4ef2a853ce5ba8e583dc

# if use_cache == False: 
#     # get data from bigquery
#     client = bigquery.Client()
#     sql = """
#     SELECT id, body, last_activity_date, score, text as comment
#     FROM `bigquery-public-data.stackoverflow.posts_answers` as answer
#     JOIN (SELECT post_id, text from `bigquery-public-data.stackoverflow.comments`) as c on c.post_id = answer.id
#     WHERE 
#     ( Date(last_activity_date) >= Date("2015-01-01")
#       AND 
#       answer.parent_id IN (
#         SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions`
#         WHERE LOWER(tags) LIKE '%python%'
#       ))

#     """
#     df = client.query(sql).to_dataframe()
#     df.to_csv("data/processed_data/stackoverflow/raw.csv")

# Read from csv files

In [26]:
%%time
# read from csv
df = pd.read_csv("data/processed_data/stackoverflow/raw9756.csv")

CPU times: user 95.5 ms, sys: 7.03 ms, total: 103 ms
Wall time: 102 ms


In [27]:
df.head()

,id,body,last_activity_date,score,comment
0,42182911,"<p>Well, since its a regular expression you co...",2017-02-12 00:32:44.407 UTC,0,"They're not mere regexes – otherwise, `<NN>+` ..."
1,46213700,"<p><a href=""https://github.com/django/django/b...",2017-09-14 17:16:22.787 UTC,0,See my note about unpacking `self.credentials`...
2,51806306,<p>You can use font attribute of worksheet col...,2018-08-12 06:25:22.257 UTC,0,"Hello, thanks for replying! After I run the co..."
3,51159568,<p>You could just keep checking each future in...,2018-07-14 17:06:02.057 UTC,0,The solution with `wait` is exactly what I was...
4,51159568,<p>You could just keep checking each future in...,2018-07-14 17:06:02.057 UTC,0,"Maybe, Except as you found out, it doesn't wor..."


In [29]:
df.shape

(9756, 5)

In [32]:
def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)


In [ ]:
# separate code and human language
